
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Sieci konwolucyjne</big></big></big></big></big>

---

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

# Sieci konwolucyjne
wykorzystują __konwolucje__ zamiast mnożenia macierzy w co najmniej jednej z warstw
* czym są _konwolucje_?
* na czym polega _rzadkość_ sieci konwolucyjnych?
* czym jest _pooling_?

## Noecognitron (Fukushima, 1980)
<img src="../nn_figures/neocognitron.png" width="70%"> ['O'Reilly]
1. na wejściu tzw. __retina__ odpowiedzialna za odbiór obrazu
2. hierarchicznie ułożone warstwy __S _oraz_ C__ 
  * __S__-warstwy (S jak simple) odpowiedzialne za ekstrakcję cech
    * zmienne wejścia modyfikowane poprzez uczenie
    * po zakończeniu uczenia każda s-komórka staje się _ekstraktorem_ pewnej konkretnej cechy w polu widzenia (reaguje na jej pojawienie się)
      * uczenie odpowiada wyborowi wykrywanej cechy
    * cechy bardziej lokalne są wykrywane bliżej wejścia modelu, te bardziej globalne później
  * __C__-warstwy (C jak complex) 
    * pozwalają na korekcję błędów translacji na wejściu
    * wejścia do C-komórek z ekstraktorów w warstwach S są ustalone i niezmienne
    * każda C-komórka dostaje wejście z grupy S-komórek wykrywających __tą samą__ cechę ale niewiele różniących się pozycjach - zapewnia inwariantność na translacje
    * C-komórka jest aktywowana jeśli co najmniej jedna S-komórka wykryła cechę
3. warstwy S i C przypominają komórki w układzie widzenia

<img src="../nn_figures/neocognitron2.gif" width="50%"> [Fukushima, ScholarPedia]
1. Sieć uczy się w procesie __samo-organizacji__
  * tylko S-komórki mają modyfikowane wagi
  * __winner-take-all__ wśród komórek w małym określonym obszarze (tzw. kolumnie) tylko jedna staje się zwycięzcą i jest aktywowana
    * połączenia zwycięzcy są wzmacniane
    * siła wzmocnienia jest proporcjonalna do aktywacji
    * na początku wszystkie połączenia są bardzo słabe
    * po jakimś czasie S-komórki uczą się rozpoznawać pewne wzorce
  * wszystkie komórki z otoczenia zwycięzcy podążają za nim
<img src="../nn_figures/neocognitron4.gif" width="70%"> [Fukushima, ScholarPedia]
2. Neocognitron osiąga na zbiorze MNIST błąd rzędu 2.5%
3. Nauczanie jest niezależne dla każdej warstwy
  * istotne stają się wielkości komórek C i S, co utrudnia uczenie

## LeNet5 (LeCun 1998)
<img src="../nn_figures/lenet5.pdf" width="100%">[LeCun NIPS]

## Konwolucje
1. dla dwu-wymiarowych obrazów $I$ można zdefiniować 2-wymiarowe kernele $K$ tak, że __dyskretna__ konwolucja będzie zdefiniowana jako
$$S(i,j)=(I*K)(i,j)=\sum_m\sum_nI(m,n)K(i-m,j-n)$$
2. konwolucje są komutatywne, skąd
$$S(i,j)=(K*I)(i,j)=\sum_m\sum_nI(i-m,j-n)K(m,n)$$
  * ten opis jest dla obrazów bardziej oczywisty 
  * zwykle implementowany implementowany będzie schemat korelacji
  $$S(i,j)=(I*K)(i,j)=\sum_m\sum_nI(i+m,j+n)K(m,n)$$
  dający ten sam wynik
  * macierz kernela jest _rzadka_: zerowa wszędzie poza polami leżącymi na obrazie
3. praktycznie można pojrzeć na obliczanie konwolucji jako mnożenie macierzy
  * konwolucje dyskretne w CNN są na ograniczonym zakresie - rozmiarze kernela

### co dają konwolucje?
* __rzadkie interakcje__
  * w sieciach warstwowych każdy element wyjściowy jest połączony z każdym wejściowym przez jakiś parametr
  * kernel jest __mniejszy__ niż wejście
  * zwiększa wydajność
  * te same cechy są wykrywane w różnych miejscach
  * w głębokich sieciach neurony w głębszych warstwach współpracują z większymi obszarami wejścia
* __współdzielenie parametrów__
  * w modelach warstwowych każda waga jest użyta __dokładnie raz__ przy liczeniu wyjścia
  * w sieci konwolucyjnej każda waga kernela jest użyta dla __każdego__ elementu wejścia (ew. poza obszarami na brzegu)
    * zamiast uczyć zestawu wag dla każdego wejścia, uczony jest __jeden__ zestaw
* __równoważne reprezentacje__
  * utworzone reprezentacje $f$ stają się odporne na translacje $t$, bo $f(t(x))=t(f(x))$
  * kernel wykrywający brzegi będzie do zastosowania w różnych miejscach obrazu
* oszczędność
  * pamięciową przez dzielenie parametrów
  * obliczeniową dzięki lokalności
* czy jednak sieci konwolucyjne mają taką samą moc obliczeniową jak sieci warstwowe na takich samych obrazach?

## Pooling
<img src="../nn_figures/cnn.pdf" width="70%"> [Goodfellow et al.]
1. typowa warstwa sieci CNN składa się z
  * szeregu __konwolucji__ tworzących liniowe aktywacje
  * __nieliniowych aktywacji__ dla wykrywania (detekcji) cech (features)
  * modyfikacji przez __pooling__
2. pooling __zastępuje__ wartość w danym miejscu pewną statystyką dla obszaru wokół
  * __max pooling__ zwraca maksymalną wartość dla pewnego prostokątnego obszaru
    * każde pole tego obszaru może odpowiadać nieco różnej wykrytej cesze
  * __average pooling__ zwraca średnią
  * __L2 norm pooling__ normą kwadratową dla prostokątnego obszaru
  * __weighted pooling__ ważoną średnią odległość od piksela w centrum obszaru
3. max pooling wprowadza pewną inwariantność na translację
  * niewielkie przesunięcie wejścia może zmienić tylko niewielką część wyjścia
  * także pewną inwariantność na powiększenie
  * pooling zmniejsza rozmiar przetwarzanego obszaru
    * w kolejnych warstwach przetwarzane są cechy wyższego poziomu
    * także efektywniejsze obliczeniowo
4. można rozróżnić dwa typy
  * pooling przestrzenny
  * pooling po kanałach
    * koncepcyjnie przypomina aktywacje _max out_ (Goodfellow)

## Konwolucje w modelach sieci neuronowych
1. konwolucja z __jednym__ kernelem wykrywa tylko __jedną__ cechę (feature), chociaż w wielu miejscach
  * w sieciach neuronowych potrzebujemy wykrywać __wiele różnych__ cech w wielu miejscach
2. wejściowy obraz składa się zwykle z wektorów obserwacji w każdym punkcie
  * np. obraz RGB
  * wejściem do konwolucji w kolejnej warstwie są wyjścia konwolucji poprzedniej 
  * dla obrazów wejścia i wyjścia są tensorami 3-D
    * jeden indeks to kanał
    * dwa indeksy podają współrzędne
  * w rzeczywistości 4-D tensory: jeszcze indeks pozycji w batchu
3. __stride__ określa przesunięcie ponad niektórymi cechami obrazu wejściowego
  * konwolucja ze stride jest równoważna pełnej konwolucji z downsamplingiem bezpośrednio później
4. __padding__ pozwala na kontrolę szerokości obrazu w kolejnych warstwach
  * bez paddingu obraz zmniejsza się co najmniej o piksel na warstwę
  * bez paddingu albo obraz szybko się zmniejsza albo potrzebne jest użycie małych kerneli
    * oba rozwiązania są niedobre
  * padding jest realizowany zwykle przez dodanie zerowych pikseli
    * możliwe także dodanie np. lustrzanego odbicia
5. możliwe alternatywy
  * __no (valid) padding__ i pixele kernela __nie mogą__ wychodzić poza obszar obrazu
    * wszystkie piksele wyjścia są funkcją __tej samej__ liczby pikseli wejścia
    * każda kolejna warstwa zmniejsza się
    
    <img width="150px" src="../nn_figures/no_padding_no_strides.gif"><img width="150px" src="../nn_figures/no_padding_strides.gif">[Dumoulin, Visin, arXiv:1603.07285]
    * liczba pikseli wyjściowych wynosi (dla $s=1$) $$o=(i-k)+2p+1$$
    * dla $s>1$ liczba pikseli wyjściowych $$o=\left\lfloor\frac{i-k}{s}\right\rfloor+1$$
  * __same (half) padding__ zapewnia tyle dodanych pikseli by warstwy __nie zmniejszały__ się
    * może być dowolna liczba warstw konwolucji
    * piksele blisko brzegu wpływają na mniej pikseli wyjściowych
    
    <img width="150px" src="../nn_figures/same_padding_no_strides.gif">[Dumoulin, Visin, arXiv:1603.07285]
    * liczba pikseli wyjściowych (dla $s=1, k=2n+1, p=\lfloor{}k/2\rfloor$) to $$o=i+2\lfloor{}k/2\rfloor-(k-1)=i+2n-2n=i$$
  * __full padding__ gdzie dodane jest tyle zer, by każdy piksel obrazu był odwiedzony tą samą liczbę razy
    * każdy __wyjściowy__ piksel blisko brzegu jest funkcją mniejszej liczby pikseli wejściowych
    
    <img width="150px" src="../nn_figures/full_padding_no_strides.gif">[Dumoulin, Visin, arXiv:1603.07285]
    * liczba pikseli wyjściowych (dla $p=k-1, s=1$) to $$o=i+2(k-1)-(k-1)=i+(k-1)$$
    
    <img width="150px" src="../nn_figures/padding_strides.gif"><img width="150px" src="../nn_figures/padding_strides_odd.gif">[Dumoulin, Visin, arXiv:1603.07285]
  * w ogólnym przypadku liczba wyjściowych to $$o=\left\lfloor\frac{i+2p-k}{s}\right\rfloor+1$$
6. zwykle optymalne rozwiązanie leży gdzieś między _valid_ a _same_

## Pooling
1. warstwy pooling zapewniają pewne niewielkie inwariancje na translacje wejścia
2. __max pooling__ 
  * dzieli wejście na patche
    * zwykle __nie__ nakładające się
  * wybraniu maksymalnej wartości
3. pooling nie wykorzystuje paddowania, a więc $$o=\left\lfloor\frac{i-k}{s}\right\rfloor+1$$
  * a więc tak samo jak konwolucje bez paddingu

## Dekonwolucje
1. a gdybyśmy chcieli odzyskać widzialne obrazy z głębokich reprezentacji?
  * prostym rozwiązaniem jest mnożenie przez __transpozycję__ macierzy konwolucji
  * modele auto-enkoderów, RBM, itp.
2. jest to jednak trochę bardziej złożone niż w modelach warstwowych
3. konwolucje __transponowane__

  <img width="150px" src="../nn_figures/no_padding_no_strides.gif"><img width="150px" src="../nn_figures/no_padding_no_strides_transposed.gif">[Dumoulin, Visin, arXiv:1603.07285]
  * konwolucja $3\times3$ nad wejsciem $4\times4$ bez paddingu ($p=0$) i jednostkowym stridem (tzn. $s=1$)
  * równoważne konwolucji $3\times3$ nad wejściem $2\times2$ wypadowane z każdej strony pasem $2$ pikseli ($p=2$) z jednostkowym strajdem ($s=1$)
  * jednak narożne piksele wejścia wpływają __jedynie__ na naróżne piksele odtwarzanego obrazu
    * paddowanie ma wymiar $p=k-1$
    * stąd wymiar wyjściowy to $o'=i'+(k-1)$
  * takie transponowane konwolucje zresztą jedynie odtwarzają __kształt__, a nie ma tu żadnej pewności odtwarzania wejścia
4. konwolucje strajdowane __cząstkowo__ (_ang_. __fractionally__)
  * czy możemy sobie wyobrazić sytuację z $s<1$??
  
  <img width="150px" src="../nn_figures/no_padding_strides_transposed.gif"><img width="150px" src="../nn_figures/padding_strides_transposed.gif">[Dumoulin, Visin, arXiv:1603.07285]
  * wyjście będzie miało wymiar $$o'=s(i'-1)+k$$ gdzie $p'=k-1$ a rozszerzone wejście $i'$ jest uzyskane przez dodanie $s-1$ pasów zer miedzy wszystkimi zerami/kolumnami

## Konwolucje rozmyte (dilated)
1. albo __atrous__ z francuskiego ___a trous___ (_z dziurami_)
2. rozmyciu ulega kernel przez dodanie spacji pomiędzy elementy jądra
  * dodawane jest $d-1$ rzędów/wierszy spacji, gdzie $d=1$ odpowiada konwolucji nierozmytej
  * rozmycie kernela sztucznie zwiększa jego wymiar do $$k'=k+(k-1)(d-1)$$
  <img width="150px" src="../nn_figures/dilation.gif">[Dumoulin, Visin, arXiv:1603.07285]

## Konwolucje tiled
1. składają się z kilku konwolucji sąsiadujących
  * konwolucje są osobne
  * mają różne parametry

## Uczenie sieci CNN
1. każdy kernel jest odpowiedzialny za wykrywanie pewnej cechy
2. okazuje się, że wsteczna propagacja jest __wystarczająca__
  * konwolucja
  * wsteczna propagacja od wyjścia do wag
  * wsteczna propagacja od wyjscia do wejścia
  
  są wystarczające dla nauczenie dowolnej sieci CNN z propagacją wprzód
  
3. __bias__ jest także elementem sieci CNN
  * w sieciach warstwowych typowy bias jest związany z każdym neuronem
  * w CNN każdy bias jest typowo związany z każdym kanałem
    * możliwe jest rozróżnienie biasu dla różnych położeń obrazu

# ImageNet: Alex net
<img src="../nn_figures/imagenet.pdf" width="80%">[Krizhevsky, Sutskever, Hinton NIPS]
1. Konkurs ILSVRC'2010 (ImageNet Large-Scale Visual Recognition Challenge)
  * oryginalny zbiór danych ma ponad 22 miliony obrazów etykietowanych ponad 22 tysiącami klas
  * zbiór danych konkursu to 1.2 miliona obrazów z 1000 różnych klas
  * 50 tysięcy walidacyjnych, 150 tysięcy testowych
  * obrazy różnych rozdzielczości przeskalowane i wycięte do $256\times256$
    * w końcowym uczeniu użyte obrazy $224\times224$
  * dwie miary błędów
    * __top-1__ binarna miara prawidłowa/nieprawidłowa
    * __top-5__ że prawidłowa etykieta __jest/nie jest__ wśród 5-ciu zaproponowanych
  * problem uczony na danych __jedynie__ z odjętą średnią dla każdego piksela/kanału
2. wyniki
  * top-1 37.5\% błędów
  * top-5 17.0\% błędów
    * jedna z wersji osiągnęła 15.3\% w top-5
    * drugi najlepszy miał w top-5 26.2\% błędów